<h1>Network Slimming</h1>
This notebook demonstrates the implementation of this paper <a href=https://arxiv.org/abs/1708.06519> Learning Efficient Convolutional Networks through Network Slimming</a>
<h4>Steps to train a baseline model and then compress it given a channel budget are as follows:</h4>
<ul>
    <li>Load the YAML file. </li>
    <li>Load dataset and create dataloaders. </li>
    <li>Create <b>Network_Slimming</b> object and pass the parameters in the form of a dictionary. </li>
    <li>Pass the dataloaders into the <b>compress_model</b> method to obtain the compressed model. </li>
</ul>
Since this is a demo notebook the number of epochs have been set to 2.

In [1]:
import sys
import os
sys.path.append("../../")
os.environ['CUDA_VISIBLE_DEVICES']='1'

import torch
from torchvision import transforms

import yaml

from trailmet.datasets.classification import DatasetFactory
from trailmet.algorithms.prune.network_slimming import Network_Slimming

In [2]:
!pwd
root = "/workspace/code/Nahush26/trail/trailmet/algorithms/network_slimming"

/workspace/code/Nahush26/trail/trailmet/experiments/network_slimming


<h3> Loading the YAML file. </h3>

In [4]:
with open(os.path.join(root, "resnet50_cifar100.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)
data_loaded['schema_root'] = root
data_loaded

{'num_classes': 100,
 'weight_decay': 0.0005,
 'net': 'resnet50',
 'dataset': 'c100',
 'epochs': 2,
 's': 0.003,
 'learning_rate': 0.002,
 'fine_tune_epochs': 2,
 'fine_tune_lr': 0.0004,
 'prune_ratio': 0.5,
 'schema_root': '/workspace/code/Nahush26/trail/trailmet/experiments/network_slimming'}

<h3>Loading CIFAR100Dataset</h3>

In [5]:
transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Pad(4, padding_mode='reflect'),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomCrop(32),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                ]
            )

transforms1 = {
    'train': transform_train, 
    'val': transform_test, 
    'test': transform_test}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = "./data",
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.1,
                                        transform = transforms1,
                                        target_transform = target_transforms,
                                        random_seed=42
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


<h5>Creating the dataloaders</h5>

In [6]:
dataloaders = { 'train' : torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    ),
               'val':  torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    ),  
               'test':  torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )}

<h3> Creating the method's object proceed with compression. </h3>

In [7]:
slim = Network_Slimming(**data_loaded)

In [9]:
slim.compress_model(dataloaders)

EPOCH NUMBER  0


100%|████████████████████████████████████████████████████████████| 79/79 [00:07<00:00, 10.13it/s, acc=0.0144, loss=4.58]


**Saving model**
EPOCH NUMBER  1


100%|████████████████████████████████████████████████████████████| 79/79 [00:07<00:00, 10.18it/s, acc=0.0172, loss=4.54]


**Saving model**
============shortcuts [0]===========
model.layer1.0.bn3: 121
model.layer1.0.downsample.1: 126
model.layer1.1.bn3: 140
model.layer1.2.bn3: 130
merged indexes length: 251
============shortcuts [1]===========
model.layer2.0.bn3: 259
model.layer2.0.downsample.1: 263
model.layer2.1.bn3: 254
model.layer2.2.bn3: 245
model.layer2.3.bn3: 267
merged indexes length: 503
============shortcuts [2]===========
model.layer3.0.bn3: 520
model.layer3.0.downsample.1: 525
model.layer3.1.bn3: 529
model.layer3.2.bn3: 545
model.layer3.3.bn3: 547
model.layer3.4.bn3: 558
model.layer3.5.bn3: 581
merged indexes length: 1013
============shortcuts [3]===========
model.layer4.0.bn3: 982
model.layer4.0.downsample.1: 982
model.layer4.1.bn3: 946
model.layer4.2.bn3: 979
merged indexes length: 1581

BatchNorm2d prune info
|    | name                        | channels   | prune percent   |
|---:|:----------------------------|:-----------|:----------------|
|  0 | model.bn1                   | 29/64      |

100%|████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 13.91it/s, acc=0.0206, loss=4.51]


**Saving model with key**
EPOCH NUMBER  1


100%|████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 13.90it/s, acc=0.0212, loss=4.46]


**Saving model with key**
